Importamos los módulos necesarios.

In [ ]:
import datetime
from ib_insync import IB, Contract, util
import pandas as pd
import time

Conectamos con IB, cambiar el puerto 4007 por el que tengas configurado en TWS.

In [1]:
util.startLoop()
ib = IB()
ib.connect('127.0.0.1', 4007, clientId=1)

<IB connected to 127.0.0.1:4003 clientId=1>

Definimos la función para descarga de datos desde Interactive Brokers por iteración, recibe un contrato y los parametros de la descarga y devuelve un Pandas DataFrame.

La función hace una parada de 5 segundos entre cada bloque de descarga para evitar superar los limites de llamadas de la API de IB. Por lo que es aconsejable ajustar el valor de **duration** lo mas alto que permita la API para cada **barSize**.

La función por seguridad va guardando los datos en un archivo csv en el mismo directorio que el Notebook. Así en caso de error o corte de conexión a mitad del proceso podemos rescatar los datos ya descargados.

In [15]:
def ib_data_download(contract, start_time=None, end_time=None, duration='5 D',
                     barSize='1 min', whatToShow='MIDPOINT', useRTH=True,
                     formatDate=1, save=True):

    if end_time:
        end_time_p = datetime.datetime.strptime(end_time, '%Y%m%d %H:%M:%S')
    else:
        end_time = datetime.datetime.now()
    if start_time:
        start_time = datetime.datetime.strptime(start_time, '%Y%m%d %H:%M:%S')
    else:
        start_time = end_time - datetime.timedelta(days=30)

    tiempo = ''
    stamp = int(time.time())
    data = pd.DataFrame()
    print('Descargado hasta :')

    while end_time > start_time:
        contract = contract
        bars = ib.reqHistoricalData(
            contract,
            endDateTime=tiempo,
            durationStr=duration,
            barSizeSetting=barSize,
            whatToShow=whatToShow,
            useRTH=useRTH,
            formatDate=formatDate,
            keepUpToDate=False)

        df = util.df(bars)
        end_time = df.iloc[0].date
        tiempo = end_time.strftime('%Y%m%d %H:%M:%S')
        print(f'   {tiempo}')
        data = data.append(df)
        if save:
            df.to_csv(f'Downloading {contract.symbol} {barSize} - {stamp}.csv')
        time.sleep(5)

    data = data.set_index('date').sort_index()[start_time:]
    return data

Descargamos como ejemplo el futuro continuo del emini SP.
Para ello definimos primero el contrato.

Solo descargará si estas suscrito a estos datos.

In [16]:
contract = Contract(symbol='ES', secType='CONTFUT', exchange='GLOBEX',
                           includeExpired=True)

Asignamos a una variable el retorno de la función, a la que pasamos el contrato, y que queremos ver la información de los Trades.

Como no le pasamos valor para los parametro start_time ni end_time, descargará los últimos 30 días.

In [17]:
df = ib_data_download(contract, whatToShow='TRADES')

Descargado hasta :
   20190228 15:30:00
   20190221 15:30:00
   20190213 15:30:00
   20190206 15:30:00
   20190130 15:30:00


Veamos el resultado.

In [18]:
df.head()

,open,high,low,close,volume,barCount,average
date,,,,,,,
2019-02-05 15:30:00,2728.00,2728.25,2726.75,2728.00,12299,1871,2727.350
2019-02-05 15:31:00,2728.25,2730.25,2727.75,2730.00,9752,1840,2729.100
2019-02-05 15:32:00,2730.25,2730.50,2728.25,2728.75,11563,1783,2729.500
2019-02-05 15:33:00,2728.50,2729.50,2728.25,2729.50,7840,1170,2728.875
2019-02-05 15:34:00,2729.50,2729.75,2728.50,2729.25,8808,1324,2729.075


Probemos ahora con el Euro/Dolar usando barras de 15 mínutos y descargando por meses, desde inicio del 2018, hasta el momento actual. 

Para este contrato no es necesaria suscripción.

In [19]:
from ib_insync import Forex

c_eurusd = Forex('EURUSD')
eurusd = ib_data_download(c_eurusd, start_time='20180101 00:00:00',
                          duration='1 M', barSize='15 mins')

Descargado hasta :
   20190205 23:15:00
   20190106 23:15:00
   20181205 23:15:00
   20181105 23:15:00
   20181007 23:15:00
   20180905 23:15:00
   20180806 23:15:00
   20180708 23:15:00
   20180606 23:15:00
   20180507 23:15:00
   20180408 23:15:00
   20180307 23:15:00
   20180205 23:15:00
   20180107 23:15:00
   20171206 23:15:00


Combrobamos el resultado.

In [20]:
eurusd.head()

,open,high,low,close,volume,barCount,average
date,,,,,,,
2018-01-02 01:00:00,1.201060,1.201285,1.200930,1.201285,-1,-1,-1.0
2018-01-02 01:15:00,1.201285,1.201780,1.201280,1.201465,-1,-1,-1.0
2018-01-02 01:30:00,1.201465,1.201500,1.201245,1.201310,-1,-1,-1.0
2018-01-02 01:45:00,1.201310,1.201660,1.201130,1.201610,-1,-1,-1.0
2018-01-02 02:00:00,1.201610,1.202305,1.201585,1.202200,-1,-1,-1.0
